# Problemas de líneas de espera

In [3]:
from random import expovariate
from statistics import mean 
from math import inf as Infinity

# **Instrucciones**

Elabore esta tarea en una notebook independiente, registrando sus resultados o las salidas de  los programas. En todos los casos en que sea posible, resuelva los problemas mediante  simulación y después verifique los resultados con las fórmulas correspondientes.  


***1. En la biblioteca***

Un encargado de la biblioteca es el único en el mostrador durante todo su turno de trabajo.  Las llegadas al mostrador siguen una distribución de Poisson con una media de $8$ por hora.  Cada usuario de la biblioteca es atendido de uno en uno, y el tiempo de servicio sigue una  distribución exponencial con una media de $5$ minutos. 


a) ¿Cuál es la probabilidad de que se forme cola? 

b) ¿Cuál es la longitud media de la cola? 

c) ¿Cuál es el tiempo medio que un cliente pasa en la biblioteca hasta que le han atendido? 

d) ¿Cuál es el tiempo medio que un cliente pasa en la cola esperando a que le atiendan? 

e) El encargado de la biblioteca pasa su tiempo en que no hay clientes clasificando artículos  de revistas. Si puede clasificar 22 fichas por hora como media cuando trabaja continuamente, ¿cuántas fichas puede ordenar durante su trabajo? 


$\textcolor{red}{Solución}$

In [29]:
def sim(lamb,mu):
    Num_Pkts = 100                   # Numero de paquetes a simular
    count    = 0                     # Contador de paquetes simulados
    t        = 0                     # Tiempo simulado
    N        = 0                     # Variable de estado: numero de paquetes en el sistema

    tll      = expovariate(lamb)     # Tiempo de llegada
    ts       = Infinity              # Tiempo de salida

    tea      = 0.0                   # tiempo del último evento


    # Variables de salida
    tll_Data = []                       # Lista de tiempos de llegada
    ts_Data  = []                       # Lista de tiempos de salida
    r_Data   = []                       # Lista de retrasos individuales
    Area     = []                       # Area de escalón


    while count < Num_Pkts:
        if tll < ts:                    # Evento de llegada
            t = tll                     
            tll_Data.append(t)          # Se guarda el tiempo de llegada
            Area.append((t - tea)*N)    # Area del escalón 
            tea = t                     # Se guarda el tiempo anterior de llegada
            N = N + 1.0                 # Se suma un paquete
            tll = t + expovariate(lamb) # Se genera un nuevo timepo de llegada
            if N == 1:
                ts = t + expovariate(mu)# Si hay un paquete en el sistema se crea el tiempo de salida
        
        else:                           # Evento de salida
            t = ts
            ts_Data.append(t)           # Se guarda el tiempo de salida
            Area.append((t - tea)*N)    # Área del escalon
            tea = t                     # Se guarda el tiempo anterior de salida
            N = N - 1.0                 # Se resta un paquete que salió
            count = count + 1           # Paquetes simulados
            if N > 0:
                ts = t + expovariate(mu)# Genera el proximo tiempo de salida
            else:
                ts = Infinity           # Si no hay paquetes en el sistema
                
    # Estimación del retraso promedio y número promedio de paquetes en el sistema:

    for i in range(Num_Pkts):
        d = ts_Data[i] - tll_Data[i]
        r_Data.append(d)

    W  = round( mean(r_Data), 4)       # Retraso promedio
    L  = round(sum(Area)/t, 4)         # Número promedio de paquetes en el sistema
    Wq = W - 1/mu                      # Tiempo medio de espera de los clientes en la cola.
    Lq = lamb * Wq                     # Número medio de clientes en la cola.

    return W,L,Wq,Lq

**Datos**

$\lambda = 8$ clientes/hora

$\mu = 12$ clientes/hora

$\rho = \frac{\lambda}{\mu} = \frac{8}{12}$

$\textcolor{blueviolet}{a)}$

La probabilidad de que se forme cola es igual a la probabilidad de que el servidor esté ocupado y eso se obtiene con la fórmula:

$$p = \frac{\lambda}{\mu}$$

In [1]:
#Parámetros
lamb = 8
mu   = 12

In [5]:
print('Probabilidad de que se forme cola: '+ str(round(lamb/mu,4)))

Probabilidad de que se forme cola: 0.6667


$\textcolor{blueviolet}{b)}$

Longitud media de la cola = $L_q$

Aquí ya usaremos simulación:

In [30]:
W  = []
L  = []
Wq = []
Lq = []
for i in range(100000): #Simulando varias veces
    lista_variables_salida = sim(lamb,mu)
    W.append(lista_variables_salida[0])
    L.append(lista_variables_salida[1])
    Wq.append(lista_variables_salida[2])
    Lq.append(lista_variables_salida[3])
    # Variables obtenidas:
    #print('Tiempo medio de estancia en el sistema: ' + str(mean(W)))
    #print('Número promedio de paquetes en el sistema: ' + str(mean(L)))
    #print('Tiempo medio de espera de los clientes en la cola: ' + str(mean(Wq)))
    #print('Número medio de clientes en la cola: ' + str(mean(Lq)))


In [31]:

print('Número medio de clientes en la cola: ' + str(mean(Lq)))

Número medio de clientes en la cola: 1.2112047893333333


$\textcolor{blueviolet}{c)}$

El tiempo medio que un cliente pasa en la biblioteca hasta que le han atendido = $W$


In [33]:
print('Tiempo medio de estancia en el sistema: ' + str(mean(W)))

Tiempo medio de estancia en el sistema: 0.234733932


$\textcolor{blueviolet}{c)}$

El tiempo medio que un cliente pasa en la cola esperando a que le atiendan = $W_q$

In [34]:
print('Tiempo medio de espera de los clientes en la cola: ' + str(mean(Wq)))

Tiempo medio de espera de los clientes en la cola: 0.15140059866666666


***2. Lavado de autos***

Un pequeño autoservicio de lavado que solo tienen capacidad para atender de uno en uno a  los autos que llegan, tiene un aparcamiento para $10$ coches, incluyendo el que está siendo lavado. La empresa ha estimado que los coches llegan siguiendo una distribución de Poisson  con una media de $20$ coches/hora, el tiempo de servicio sigue una distribución exponencial  de $12$ minutos. La empresa abre durante $10$ horas al día.  

¿Cuál es la media de coches perdidos cada día debido a las limitaciones de espacio?


$\lambda = 20$ Coches/hora

$\mu= 5 $ Coches/hora

$T=10$ Horas

$L= 10$ Longitud de la cola

In [4]:
def sim(lamb,mu,T,L):
    count    = 0                     # Contador de paquetes simulados
    t        = 0                     # Tiempo simulado
    N        = 0                     # Variable de estado: numero de paquetes en el sistema

    tll      = expovariate(lamb)     # Tiempo de llegada
    ts       = Infinity              # Tiempo de salida

    tea      = 0.0                   # tiempo del último evento
    perdidos = 0                     # Clientes perdidos por la limitación

    # Variables de salida
    tll_Data = []                       # Lista de tiempos de llegada
    ts_Data  = []                       # Lista de tiempos de salida
    r_Data   = []                       # Lista de retrasos individuales
    Area     = []                       # Area de escalón


    while t < T:                        # T = tiempo total a simular
        if tll < ts:                    # Evento de llegada
            t = tll                     
            tll_Data.append(t)          # Se guarda el tiempo de llegada
            Area.append((t - tea)*N)    # Area del escalón 
            tea = t                     # Se guarda el tiempo anterior de llegada
            if N <= L:
                N = N + 1.0             # Se suma un paquete
            else: 
                perdidos += 1           # Se cuentan los clientes que se van
            tll = t + expovariate(lamb) # Se genera un nuevo timepo de llegada
            if N == 1:
                ts = t + expovariate(mu)# Si hay un paquete en el sistema se crea el tiempo de salida
        
        else:                           # Evento de salida
            t = ts
            ts_Data.append(t)           # Se guarda el tiempo de salida
            Area.append((t - tea)*N)    # Área del escalon
            tea = t                     # Se guarda el tiempo anterior de salida
            N = N - 1.0                 # Se resta un paquete que salió
            count = count + 1           # Paquetes simulados
            if N > 0:
                ts = t + expovariate(mu)# Genera el proximo tiempo de salida
            else:
                ts = Infinity           # Si no hay paquetes en el sistema
                
    # Estimación del retraso promedio y número promedio de paquetes en el sistema:

    for i in range(T):
        d = ts_Data[i] - tll_Data[i]
        r_Data.append(d)

    W  = round( mean(r_Data), 4)       # Retraso promedio
    L  = round(sum(Area)/t, 4)         # Número promedio de paquetes en el sistema
    Wq = W - 1/mu                      # Tiempo medio de espera de los clientes en la cola.
    Lq = lamb * Wq                     # Número medio de clientes en la cola.

    return W,L,Wq,Lq,perdidos

In [10]:
lista_perdidos = []
for i in range(100000):
    ej2 = sim(20,5,10,10)
    lista_perdidos.append(ej2[4])
print('Número medio de clientes perdidos: ' + str(mean(lista_perdidos)))


Número medio de clientes perdidos: 140.24421


***3. John Macko***

John Macko es alumno en la U de Ozark. Hace trabajos extraños para aumentar sus ingresos. Las peticiones de trabajo llegan en promedio cada 5 días, pero el tiempo entre ellas es  exponencial. El tiempo para terminar un trabajo también es exponencial, con una media de 4  días. 

a) ¿Cuál es la probabilidad de que le falte trabajo a John? 

b) Si John cobra unos $50 por cada trabajo, ¿cuál es su ingreso mensual promedio? 

c) Si al final del semestre John decide subcontratar los trabajos pendientes a $40 cada uno, ¿cuánto debe esperar pagar en promedio?
